In [ ]:
!pip install datasets
!pip install transformers==4.25.1
!pip install Rouge

In [ ]:
#라이브러리 임포트

import datasets
import transformers
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import itertools
import re
import torch
from rouge import Rouge
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    EarlyStoppingCallback

)
from datasets import Dataset
from tqdm import tqdm
import glob
import json
import os

In [ ]:
#데이터 로드
train = pd.read_csv('/content/drive/MyDrive/카카오톡일상대화/train_new_2.csv')
valid = pd.read_csv('/content/drive/MyDrive/카카오톡일상대화/valid_new_2.csv')

In [ ]:
def preprocess_sentence(sentence):
    sentence = sentence.lower() # 텍스트 소문자화
    sentence = re.sub(r'[ㄱ-ㅎㅏ-ㅣ]+[/ㄱ-ㅎㅏ-ㅣ]', '', sentence) # 여러개 자음과 모음을 삭제한다.
    sentence = re.sub("[^가-힣a-z0-9#@,-]", " ", sentence) # 영어 외 문자(숫자, 특수문자 등) 공백으로 변환
    sentence = re.sub(r'[" "]+', " ", sentence) # 여러개 공백을 하나의 공백으로 바꿉니다.
    sentence = sentence.strip() # 문장 양쪽 공백 제거
    
    return sentence

def data_process(data):
  # 전체 Text 데이터에 대한 전처리 (1)
  text = []

  for data_text in tqdm(data):
    text.append(preprocess_sentence(data_text))
  
  return text

In [ ]:
# 데이터 가공
train_texts = data_process(train['Text'])
val_texts = data_process(valid['Text'])

train_df = pd.DataFrame(zip(train_texts,train['Summary']), columns=['Text', 'Summary'])
val_df = pd.DataFrame(zip(val_texts,valid['Summary']), columns=['Text', 'Summary'])

In [ ]:
# DF > data Set으로 전환
train_data = Dataset.from_pandas(train_df) 
val_data = Dataset.from_pandas(val_df)
test_samples = Dataset.from_pandas(val_df)

print(train_data)
print(val_data)
print(test_samples)

In [ ]:
#model_checkpoints = "/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/checkpoint/domain_adaptation/checkpoint-12500"


# 사용하고자 하는 허깅페이스의 모델
model_checkpoints = 'gogamza/kobart-base-v2'
#model_checkpoints = 'psyche/KoT5-summarization'


tokenizer = AutoTokenizer.from_pretrained(model_checkpoints)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

In [ ]:
# max_input = 300
# max_target = 100

# 기존
max_input = 256
max_target = 64

# 기존
# max_input = 512
# max_target = 128



ignore_index = -100# tokenizer.pad_token_id

def add_ignored_data(inputs, max_len, ignore_index):
  if len(inputs) < max_len:
      pad = [ignore_index] *(max_len - len(inputs))  # ignore_index즉 -100으로 패딩을 만들 것인데 max_len - lne(inpu)
      inputs = np.concatenate([inputs, pad])         # max_input만큼 -100으로 남은 공간을 패딩처리
  else:
      inputs = inputs[:max_len]                      # 최대 길이보다 인풋이 크다면 최대길이만큼만 가져오기

  return inputs

def add_padding_data(inputs, max_len):
    pad_index = tokenizer.pad_token_id              # 입력의 크기를 동일하게 하기 위한 pad 토큰들의 인덱스 지정
    if len(inputs) < max_len:
        pad = [pad_index] *(max_len - len(inputs))
        inputs = np.concatenate([inputs, pad])
    else:
        inputs = inputs[:max_len]

    return inputs 

def preprocess_data(data_to_process):
    label_id= []
    label_ids = []
    dec_input_ids = []
    input_ids = []
    bos = tokenizer('')['input_ids']
    for i in range(len(data_to_process['Text'])):
        input_ids.append(add_padding_data(tokenizer.encode(data_to_process['Text'][i], add_special_tokens=False), max_input))
    for i in range(len(data_to_process['Summary'])):
        label_id.append(tokenizer.encode(data_to_process['Summary'][i]))  
        label_id[i].append(tokenizer.eos_token_id)   
        dec_input_id = bos
        dec_input_id += label_id[i][:-1]
        dec_input_ids.append(add_padding_data(dec_input_id, max_target))  
    for i in range(len(data_to_process['Summary'])):
        label_ids.append(add_ignored_data(label_id[i], max_target, ignore_index))
   
    return {'input_ids': input_ids,
            'attention_mask' : (np.array(input_ids) != tokenizer.pad_token_id).astype(int),
            'decoder_input_ids': dec_input_ids,
            'decoder_attention_mask': (np.array(dec_input_ids) != tokenizer.pad_token_id).astype(int),
            'labels': label_ids}

train_tokenize_data = train_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])
val_tokenize_data = val_data.map(preprocess_data, batched = True, remove_columns=['Text', 'Summary'])

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model) # 데이터 일괄 처리?

In [ ]:
!pip install optuna

In [ ]:
#여기서 output_dir이거 따로 설정
#logging_dir 이것도 따로 바꿔야 함


import optuna

def objective(trial: optuna.Trial):
    model = model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)
    
    training_args = Seq2SeqTrainingArguments(
    output_dir='/content/drive/MyDrive/카카오톡일상대화/results',
    
    
    num_train_epochs=3,  # demo
       
    do_train=True,
    do_eval=True,
    
    #배치사이즈를 128,256으로 할떄는 여전히 안됨
    #16, 32
    
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=32,
    
    # learning_rate=trial.suggest_loguniform("learning_rate", low=4e-5, high=0.01),
    learning_rate=trial.suggest_categorical('learning_rate', [3e-04,3e-05,3e-06]),
    # weight_decay=trial.suggest_loguniform("weight_decay", 4e-5, 0.01),
    weight_decay=trial.suggest_categorical("weight_decay", [0.1,0.2,0.01]),
    
    predict_with_generate=True, # 생성기능을 사용하고 싶다고 지정한다.
    ##logging_dir="/content/drive/MyDrive/인공지능/생성요약프로젝트/Model/KoBART/logs2",
    logging_dir='/content/drive/MyDrive/카카오톡일상대화/logs',
    
    save_total_limit=3,
    load_best_model_at_end = True,
    logging_strategy = 'epoch',
    evaluation_strategy  = 'epoch',
    save_strategy ='epoch',
    )
        
    trainer = Seq2SeqTrainer(
    model, 
    training_args,
    train_dataset=train_tokenize_data,
    eval_dataset=val_tokenize_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    )
    result = trainer.train()
    return result.training_loss

In [ ]:
#여기가 하이퍼 튜닝 하는곳 시간이 오래 걸린다

study = optuna.create_study(study_name="hyper-parameter-search", direction="minimize")

#기존 시도는 15
study.optimize(func=objective, n_trials=3)
print(study.best_value)
print(study.best_params)
print(study.best_trial)

In [ ]:
optuna.visualization.plot_param_importances(study)

In [ ]:
optuna.visualization.plot_optimization_history(study)